In [1]:

import pandas as pd
import numpy as np

#internet package

import urllib3
from bs4 import BeautifulSoup  
http = urllib3.PoolManager()

#functional package
import logging
import requests
from newspaper import Article
from newspaper import fulltext
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

logging.basicConfig(filename="crawl.log",level=logging.ERROR)

In [3]:
def get_new_data(symbol):
    r = http.request('GET', 'http://s.cafef.vn/Ajax/Events_RelatedNews_New.aspx?symbol=' + str(symbol) + '&floorID=0&configID=0&PageIndex=1&PageSize=10000&Type=2')
    soup = BeautifulSoup(r.data, "html.parser")
    data = soup.find("ul", {"class": "News_Title_Link"})
    raw = data.find_all('li')
    data_dicts = []
    for row in raw:
        row_dict = {}
        row_dict['newsdate'] = row.span.text
        row_dict['title'] = row.a.text
        row_dict['url']  = "http://s.cafef.vn/" + str(row.a['href'])
        row_dict['ticker'] = str(symbol)
        data_dicts.append(row_dict)
    return data_dicts

In [3]:
# pd.DataFrame(get_new_data("VNM")).head()

In [4]:
public_company_list = pd.read_csv("public_company_list.csv", skiprows = 1, usecols = range(1,5))
public_company_list = public_company_list[public_company_list["SÀN"] != "OTC"]
public_company_list = public_company_list.dropna()

In [5]:
public_company_list['MÃ CK'].values

array(['A32', 'AAA', 'AAM', ..., 'YRC', 'YSC', 'YTC'], dtype=object)

In [6]:
all_datas = pd.DataFrame()
for ticker in tqdm(public_company_list['MÃ CK'].values):

    tickernews = pd.DataFrame(get_new_data(ticker))
    all_datas = pd.concat([all_datas, tickernews], ignore_index=True)

# to export data only 
# all_datas.to_csv("news_url.csv")
    

100%|██████████| 1711/1711 [26:19<00:00,  2.64it/s] 


In [13]:
all_datas.info

<bound method DataFrame.info of Empty DataFrame
Columns: []
Index: []>

In [2]:
news_url = pd.read_csv("news_url.csv", usecols = range(1,5))

In [9]:
news_url.head()['url'].values

array(['http://s.cafef.vn//a32-314278/a32-ong-nguyen-the-anh-pho-chu-tich-hdqt-da-mua-2800-cp.chn',
       'http://s.cafef.vn//a32-312903/a32-ong-nguyen-the-anh-pho-chu-tich-hdqt-dang-ky-mua-2800-cp.chn',
       'http://s.cafef.vn//a32-309371/diem-danh-nhung-doanh-nghiep-chot-quyen-nhan-co-tuc-bang-tien-bang-co-phieu-va-co-phieu-thuong-tuan-nay.chn',
       'http://s.cafef.vn//a32-308683/lich-chot-quyen-nhan-co-tuc-bang-tien-cua-8-doanh-nghiep.chn',
       'http://s.cafef.vn//a32-308769/a32-662019-ngay-gdkhq-tra-co-tuc-bang-tien-mat-700dcp.chn'],
      dtype=object)

In [3]:
news_url.head()

newsdate ticker                                              title  \
0  10/07/2019 09:34    A32  A32: Ông Nguyễn Thế Anh - Phó Chủ tịch HĐQT đã...   
1  01/07/2019 14:21    A32  A32: Ông Nguyễn Thế Anh - Phó Chủ tịch HĐQT đă...   
2  03/06/2019 08:55    A32  Điểm danh những doanh nghiệp chốt quyền nhận c...   
3  28/05/2019 13:46    A32  Lịch chốt quyền nhận cổ tức bằng tiền của 8 do...   
4  28/05/2019 13:27    A32  A32: 6.6.2019, ngày GDKHQ trả cổ tức bằng tiền...   

                                                 url  
0  http://s.cafef.vn//a32-314278/a32-ong-nguyen-t...  
1  http://s.cafef.vn//a32-312903/a32-ong-nguyen-t...  
2  http://s.cafef.vn//a32-309371/diem-danh-nhung-...  
3  http://s.cafef.vn//a32-308683/lich-chot-quyen-...  
4  http://s.cafef.vn//a32-308769/a32-662019-ngay-...

In [7]:
news_url.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330851 entries, 0 to 330850
Data columns (total 4 columns):
newsdate    330851 non-null object
ticker      330851 non-null object
title       330849 non-null object
url         330851 non-null object
dtypes: object(4)
memory usage: 10.1+ MB


In [50]:
url = "http://s.cafef.vn//a32-308769/a32-662019-ngay-gdkhq-tra-co-tuc-bang-tien-mat-700dcp.chn"

In [4]:

def get_details_from_url(df_to_get):
    data_dicts = []
    for newsdate, url in tqdm_notebook(zip(df_to_get['newsdate'].values, df_to_get['url'].values)):
        row_dict = {}
        try:
            article = Article(url, language='vi')
            article.download()
            article.parse()
            article.nlp()
            row_dict['newsdate'] = newsdate
            row_dict['summary'] = article.summary
            row_dict['keywords'] = article.keywords
            data_dicts.append(row_dict)
        except Exception as e:
#             logging.error()
            continue
#         print(data_dicts)
    return data_dicts
         

In [5]:
news_detail = pd.DataFrame(get_details_from_url(news_url))

KeyboardInterrupt: 

Get all the news with full text and summarize and keywords

In [7]:
from multiprocessing.dummy import Pool as ThreadPool 
pool = ThreadPool(32)
results = pool.map(get_details_from_url(news_url))
pool.close() 
pool.join()

KeyboardInterrupt: 